In [15]:
import os
import csv

csv_input_file = r"C:\Users\hmane\Desktop\folders\memes\ct_data\ig_fb_combined_race_lgbtq_01012016_07312023.csv"
base_output_file = 'CT_annotations_batch_{}-{}.csv'
image_base_folder_path = r"C:\Users\hmane\Downloads\split_images"  # This is the directory containing batch folders
github_base_url = "https://raw.githubusercontent.com/heranmane/label_studio_images/main/"


In [ ]:
import os
import shutil

source_dir = "/content/drive/MyDrive/Memes Project Aim 1/CT Dataset/downloaded_images_hm/img3_hm/"
target_base_dir = "/content/drive/MyDrive/Memes Project Aim 1/CT Dataset/downloaded_images_hm/split_images/"

# Ensure the target base directory exists
if not os.path.exists(target_base_dir):
    os.makedirs(target_base_dir)

# Get a list of all files
try:
    files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]
    batch_size = 100  # Adjust size to manage load
    for i in range(0, len(files), batch_size):
        batch = files[i:i+batch_size]
        batch_dir = os.path.join(target_base_dir, f'batch_{i//batch_size + 1}')
        print(batch_dir)
        if not os.path.exists(batch_dir):
            os.makedirs(batch_dir)
        for f in batch:
            shutil.move(os.path.join(source_dir, f), batch_dir)
    print("Images successfully split into batches.")
except Exception as e:
    print(f"Failed to organize images: {str(e)}")

In [16]:
# Read original CSV to get the text associated with images
text_data = {}
with open(csv_input_file, 'r', newline='', encoding='utf-8') as infile:
    csv_reader = csv.DictReader(infile)
    for row in csv_reader:
        post_url = row['URL']
        post_id = post_url.split("/")[-1]
        text_data[post_id] = row['Message']

# Process batches
batch_folders = [f for f in os.listdir(image_base_folder_path) if f.startswith('batch_')]
batch_folders.sort(key=lambda x: int(x.split('_')[1]))  # Sort folders numerically by batch number

# Accumulate images from three batches in one CSV, except the last which may have fewer
for i in range(0, len(batch_folders), 3):
    current_batches = batch_folders[i:i+3]
    if not current_batches:
        continue

    # Determine output CSV file name
    if len(current_batches) == 3:
        output_csv_file = base_output_file.format(current_batches[0], current_batches[2])
    else:
        output_csv_file = base_output_file.format(current_batches[0], current_batches[-1])

    with open(output_csv_file, 'w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['batch', 'image', 'text']
        csv_writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        csv_writer.writeheader()

        # Process each batch folder
        for batch in current_batches:
            batch_path = os.path.join(image_base_folder_path, batch)
            for image_filename in os.listdir(batch_path):
                post_id = image_filename.split('-')[1].split('.')[0]
                if post_id in text_data:
                    image_url = github_base_url + os.path.join(batch, image_filename).replace('\\', '/')
                    new_row = {
                        'batch': batch,
                        'image': image_url,
                        'text': text_data[post_id]
                    }
                    csv_writer.writerow(new_row)

In [17]:
import csv

# List of CSV files uploaded for annotation
csv_files = [
    "CT_annotations_batch_1-3.csv",
    "CT_annotations_batch_4-6.csv",
    "CT_annotations_batch_7-9.csv",
    "CT_annotations_batch_10-12.csv",
    "CT_annotations_batch_13-15.csv",
    "CT_annotations_batch_16.csv"
]

# Annotators information (example names/IDs)
annotators = [("Annotator1", "Annotator2", "TieBreaker")]

# Create the tracking CSV
tracking_csv = "annotation_tracking.csv"
with open(tracking_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["CSV File Name", "Annotator 1", "Annotator 1 Status", 
                     "Annotator 2", "Annotator 2 Status", "Tie Breaker", 
                     "Tie Breaker Status", "Overall Status"])
    
    # Write initial data
    for csv_file in csv_files:
        for annotator1, annotator2, tie_breaker in annotators:
            # Initial statuses can be adjusted as needed
            writer.writerow([csv_file, annotator1, "Not Started", annotator2, 
                             "Not Started", tie_breaker, "Not Started", "Incomplete"])

print("Tracking CSV created:", tracking_csv)


Tracking CSV created: annotation_tracking.csv


In [14]:
import pandas as pd 
df = pd.read_csv(r'C:\Users\hmane\Downloads\split_images\CT_annotations_batch_batch_1-batch_3.csv')
df

,batch,image,text
0,batch_1,https://raw.githubusercontent.com/heranmane/Me...,BLM Roundup & Ended Up at a Kill Lot in Texas!...
1,batch_1,https://raw.githubusercontent.com/heranmane/Me...,".Cherokee Speedway, we’re coming for ya! 👊 The..."
2,batch_1,https://raw.githubusercontent.com/heranmane/Me...,Marsha “Pay it No Mind” Johnson (1945-1992) wa...
3,batch_1,https://raw.githubusercontent.com/heranmane/Me...,🚨Taco Tuesday 🚨🌮🦐🌮 🚨2X1 Tacos with a Drink pur...
4,batch_1,https://raw.githubusercontent.com/heranmane/Me...,"Jacks Fam, please join us in welcoming Raegen ..."
...,...,...,...
295,batch_3,https://raw.githubusercontent.com/heranmane/Me...,Chili Cornbread Casserole https://ed.gr/duszu:...
296,batch_3,https://raw.githubusercontent.com/heranmane/Me...,HIGH WIND WARNING THRU 7 AM: A storm complex i...
297,batch_3,https://raw.githubusercontent.com/heranmane/Me...,The weather team is tracking a winter storm sy...
298,batch_3,https://raw.githubusercontent.com/heranmane/Me...,✨ NEW PRODUCT ALERT! ✨ We need your vote becau...


In [7]:
df2 = pd.read_csv(r'C:\Users\hmane\Downloads\split_images\CT_annotations_batch_batch_3-batch_5.csv')
df2

,batch,image,text
0,batch_3,https://raw.githubusercontent.com/heranmane/Me...,City officials expect an 80% chance of rain on...
1,batch_3,https://raw.githubusercontent.com/heranmane/Me...,Reposted from @prparadenyc We are currently pl...
2,batch_3,https://raw.githubusercontent.com/heranmane/Me...,"""We seek the history of Jewish women out of ou..."
3,batch_3,https://raw.githubusercontent.com/heranmane/Me...,The big guy got Caught.
4,batch_3,https://raw.githubusercontent.com/heranmane/Me...,On this special day honoring the 100th birthda...
...,...,...,...
295,batch_5,https://raw.githubusercontent.com/heranmane/Me...,Now introducing Iwate Prefecture! Iwate is kno...
296,batch_5,https://raw.githubusercontent.com/heranmane/Me...,Ever wondered about different Eggplants? Here'...
297,batch_5,https://raw.githubusercontent.com/heranmane/Me...,NaN
298,batch_5,https://raw.githubusercontent.com/heranmane/Me...,"Hey Folks! On Sunday, January 9 at 8pm ET, I w..."
